In [15]:
#importing the required libs
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


In [16]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


In [17]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

In [18]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [19]:
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# Define Foursquare Credentials and Version

In [20]:
CLIENT_ID = 'AMNJ3OIRNDFAHNNQCYTKEBG000WHPJQFSOYKL2BFY2YYGT5U' # your Foursquare ID
CLIENT_SECRET = 'TQJUJL1YF1JLOFEF51IJIF2QYUDUUDNU4DJLQUGBRZMF1W0R' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AMNJ3OIRNDFAHNNQCYTKEBG000WHPJQFSOYKL2BFY2YYGT5U
CLIENT_SECRET:TQJUJL1YF1JLOFEF51IJIF2QYUDUUDNU4DJLQUGBRZMF1W0R


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [28]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

GeocoderTimedOut: Service timed out

# search for a specific venue category

#### lets assume we are lookingfor italian food shops within 500 meters 

In [ ]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

# Defyining the url

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

#### Send the GET Request and examine the results

In [ ]:
results = requests.get(url).json()
results

###### Get relevant part of JSON and transform it into a pandas dataframe

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

#### Defining information of interest which should be displayed  and filter dataframe

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered #this data frame will be filtering location coordinatesinto readable adresses

#### Let's visualize the Italian restaurants that are nearby

In [ ]:
dataframe_filtered.name

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.Marker(
    [latitude, longitude],
    radius=10,
    icon=folium.Icon(color='red', icon='info-sign'),
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.Marker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## 2. Explore a Given Venue

###### https://api.foursquare.com/v2/venues/VENUE_ID?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION

###### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_

In [ ]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

##### Sending GET request for results

In [ ]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

### B. Get the venue's overall rating

In [ ]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

In [ ]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has beennot rated yet lets check the third one 

In [ ]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This has better rating (Ecco restraurant) lets take this forward

In [ ]:
### C. Get the number of tips

In [ ]:
result['response']['venue']['tips']['count']

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [ ]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

#### Get tips and list of associated features

In [ ]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

#### Format column width and display all tips

In [ ]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

#### Define URL, send GET request and display features associated with user

In [ ]:
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()